In [ ]:
from collections import Counter

import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

# avoid burning my eyes @ night
plt.style.use("dark_background")

In [ ]:
FILE = "data/survey_results_public.csv"
so_df = pd.read_csv(FILE)

print(so_df.keys())
so_df.describe()

# check for people who aren't paying attention
count_not_apple =  (so_df["Check"] != "Apples").sum()
print(count_not_apple)
print(so_df.shape)
assert(count_not_apple == 0)
# print(so_df[:3])


In [ ]:
# get popularity of different programming languages

#keys re: languages are:
#LanguageHaveWorkedWith,LanguageWantToWorkWith,LanguageAdmired,LanguageDesired

# draw as strip chart
# https://seaborn.pydata.org/generated/seaborn.stripplot.html#seaborn.stripplot

def get_langs(dataset, key="LanguageHaveWorkedWith"):
    lang_count = Counter()
    assert(key in dataset.keys())
    for response in dataset[key]:
        if type(response) == str:
            lang_count.update(response.split(';'))
    langs_by_popularity = dict(
        sorted(lang_count.items(), key=lambda item: item[1], reverse=True)
    )
    return langs_by_popularity

def visualize_langs(langs, langs2, label1 = "condition1", label2 = "condition2", saveto=None):
    DOT_COLOR1 = "lightblue"
    DOT_COLOR2 = "red"
    BG_COLOR   = "black" 
    df    = pd.DataFrame(langs.items(), columns=['Languages', 'Count'])
    df2   = pd.DataFrame(langs2.items(), columns=['Languages', 'Count'])
    
    plt.figure(figsize=(10,15)) 
    
    sb.stripplot(x='Count', y='Languages', data=df, \
                 size=5, color=DOT_COLOR1, label="have worked with", jitter=True)
    sb.stripplot(x='Count', y='Languages', data=df2, \
                 size=5, color=DOT_COLOR2, label="want to work with", jitter=True)
    
    # chatgpt draws my legend
    # Create custom legend handles to avoid duplicates
    # color = 'w' means do not draw line bissecting point
    blue_patch = plt.Line2D(
        [0], [0], marker='o', color=BG_COLOR, \
        label=label1, markerfacecolor=DOT_COLOR1, markersize=10)
    red_patch = plt.Line2D(
        [0], [0], marker='o', color=BG_COLOR, \
        label=label2, markerfacecolor=DOT_COLOR2, markersize=10)
    
    # Show the legend with custom handles
    plt.legend(handles=[blue_patch, red_patch], loc="center right")
    
    plt.grid(axis='x', linestyle='--', alpha=0.75) 
    plt.title("%s vs %s" % (label1, label2))
    if saveto is not None:
        plt.savefig(saveto, bbox_inches='tight')
    del df, df2

l1 = get_langs( so_df )
l2 = get_langs( so_df, "LanguageWantToWorkWith" )
visualize_langs(l1,l2, 
                label1="have worked with", label2="want to work with",
                saveto="images/used-vs-want2use.png")

l3 = get_langs( so_df, "LanguageAdmired")
l4 = get_langs( so_df, "LanguageWantToWorkWith")
visualize_langs(l3, l4, 
                label1="admired", label2="want to work with",
               saveto="images/admired-vs-want2use.png")
    

In [ ]:
# draw horizontal bar plot
# https://seaborn.pydata.org/examples/part_whole_bars.html

# investigate extrinsic vs intrinsic motivation
def get_difference(dict1, dict2):
    keys = dict1.keys()
    result = dict()
    for key in keys:
        result[key] = dict1[key] - dict2[key]
    return result

def visualize_diff(diff_dict, color="lightblue", saveto=None):
    diff_sorted = dict(
        sorted(diff_dict.items(), key=lambda item: item[1], reverse=True)
    )
    KEY = "Value"
    df    = pd.DataFrame(diff_sorted.items(), columns=['Languages', 'Value'])
    plt.figure(figsize=(15,20)) 
    sb.barplot(x=KEY, y='Languages', data=df, color=color)
    
    DELTA =  '\u0394'
    for index, value in enumerate(df[KEY]):
    # chatgpt annotates my chart
    # Position the text at the base of the bar
        if value >= 0:
            # Adjust the x position for positive values
            plt.text(value, index, DELTA+str(value), va='center', ha="left")  
        else:
             # Adjust the x position for negative values
            plt.text(value, index,  DELTA+str(value), va='center',  ha='right') 
    lowest = 0
    offset = 0.5
    positive_values = df[df[KEY] > 0][KEY]
    if not positive_values.empty:
        lowest = positive_values.min()
    if len(positive_values) < len(df):
        # don't draw the line if every value is greater than 0
        plt.axhline(y=df[KEY].tolist().index(lowest) + offset, color='red', linestyle='--')
    if saveto is not None:
        plt.savefig(saveto, bbox_inches='tight')
    
motiv_diff = get_difference(l2, l1)
# print(motiv_diff)
visualize_diff(motiv_diff, saveto="images/delta.png")

# determine level of hype
hype = get_difference(l4, l3)
# print(hype)
visualize_diff(hype, color="red")

In [ ]:
# see how much money are people making

def get_mean_by_category(df, category, key="ConvertedCompYearly"):
    unique = df[category].unique()
    result = dict()
    for u in unique:
        mean = df[df[category] == u][key].mean()
        result[u] = mean
    return result

def show_me_the_money(df, saveto=None):
    key_x = "ConvertedCompYearly"
    key_y = "DevType"
    
    means   = get_mean_by_category(df, key_y) 
    mean_df = pd.DataFrame(means.items(), columns=[key_y, key_x])

    plt.figure(figsize=(14,18)) 
    plt.axvline(x=1e5, color='red', linestyle='--', label="x = $100,000")
    plt.axvline(x=1e6, color='lightgreen', linestyle='--', label="x = millionaire")
    sb.barplot(x=key_x, y=key_y, data=mean_df.sort_values(by=key_x), \
               color='lavender', alpha=0.7, label="average compensation")
    sb.stripplot(x=key_x, y=key_y, data=df, \
                 size=3, jitter=True)
    if saveto is not None:
        plt.savefig(saveto, bbox_inches='tight')
    
# print survey ans
#employment_status = Counter(so_df["MainBranch"])
#print(employment_status)

#employment_type = Counter(so_df["DevType"])
#print(employment_type)

key = "ConvertedCompYearly"
#    answers = so_df[:-1][key].count()
#    print(answers, "people answered re: ", key)
df_no_na = so_df.dropna(subset=[key])
indices  = df_no_na[key].nlargest(15).index

show_me_the_money( df_no_na.drop(indices), saveto="images/compensation-by-profession.png" )
# could also ask myself what portion of developers 
# earn less than the mean compensation
# (what titles have high standard deviations in earnings)